In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
# Step 1: Load the Stanford Dogs dataset
(train_data, test_data), info = tfds.load('stanford_dogs', split=['train', 'test'], with_info=True, as_supervised=True)

In [3]:
# Step 2: Preprocess the data
IMG_SIZE = (224, 224)
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_data = train_data.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [4]:
# Step 3: Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
# Step 4: Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

In [6]:
# Step 5: Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Step 6: Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Step 7: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Step 8: Train the top layers
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history = model.fit(train_data, epochs=10, validation_data=test_data, callbacks=callbacks)

Epoch 1/10


2025-01-09 10:19:41.030809: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0081 - loss: 4.7957

KeyboardInterrupt: 

## Adafilter Implementation

In [ ]:
# Step 8: Fine-Tune the Important Filters (AdaFilter Implementation)

# 1. Identify Important Filters:

import numpy as np

def get_important_filters(model, layer_name, top_n=10):
    layer = model.get_layer(name=layer_name)
    weights, _ = layer.get_weights()
    importance_scores = np.mean(np.abs(weights), axis=(0, 1, 2))
    important_filters = np.argsort(importance_scores)[-top_n:]
    return important_filters

In [ ]:
# 2. Fine-Tune Important Filters:

# Assuming 'conv4_block1_1_conv' is a layer name you want to fine-tune
important_filters = get_important_filters(model, 'conv4_block1_1_conv', top_n=10)

for i, layer in enumerate(base_model.layers):
    if 'conv' in layer.name:
        if i in important_filters:
            layer.trainable = True
        else:
            layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              

In [ ]:
# Step 9: Fine-Tune the Model

history_fine_tune = model.fit(train_data,
                              epochs=10,
                              validation_data=test_data,
                              callbacks=callbacks)

In [ ]:
# Step 10: Evaluate the Model

loss, accuracy = model.evaluate(test_data)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')